# Pair-rule gene mutant simulations

In [12]:
import sys
sys.path.insert(0, '../functions/')

from imputation import *
from data_prep import *

## Load (x,y,z)-coordinates

In [13]:
raw_data = np.loadtxt('../data/dmel_data.txt').T
xyz = raw_data[:,:3]

In [14]:
def make_quad(X):
    quad = np.zeros((int(X.shape[0]), int(X.shape[1] + (X.shape[1]*(X.shape[1]-1))/2)))
    quad[:, :X.shape[1]] = np.copy(X)
    col = 99
    for i in range(X.shape[1]-1):
        for j in range(i+1, X.shape[1]):
            quad[:,col] = (X[:,i]*X[:,j])
            col += 1
    return quad

## Load gene name pickle and create a dictionary

In [15]:
with open('../pickles/names.pkl', 'rb') as f:
    names = pickle.load(f)
    
name_dic = {}
for (i,name) in enumerate(names):
    name_dic[name] = i

## Load complete data with imputated values
This data set is not uploaded in the repository as the file size exceeds the limit. Before running the cells below, be sure to run the notebook 1. Data preparation to obtain a complete data set.

## Simulations

In [ ]:
data = np.loadtxt('../data/data_complete.txt')
init = np.vstack((np.vsplit(data, 6)[0]))

w = res['w'][-1]
bias = res['bias'][-1]

genes = [name_dic['h'], name_dic['eve'], name_dic['ftz'], name_dic['odd'], name_dic['prd'], name_dic['slp1']]
fig = plt.figure(figsize=(6,4.5), dpi=300)
fig.subplots_adjust(wspace=0.15, hspace=0.5)
for (e_s, source) in enumerate(genes):
    prop = 1
    if e_s == 0:
        true = np.vstack((np.vsplit(data, 6)[prop+1]))
        quad = make_quad(init)
        f = init + quad.dot(w) + bias
        f[f<0] = 0
        
        for i in range(prop):
            quad = make_quad(f)
            f = f + quad.dot(w) + bias
            f[f<0] = 0
        
        for (e,target) in enumerate(genes):
            norm = plt.Normalize(np.min(data[:,target]), np.max(data[:,target]))
            ax = fig.add_subplot(8,6,e+1)
            ax.axis('off')
            ax.scatter(xyz[:,0][xyz[:,1] > 0], xyz[:,2][xyz[:,1] > 0],
                      c=true[:,target][xyz[:,1] > 0], s=0.5, norm=norm,
                      cmap=plt.cm.Blues)
            ax.text(0,120,names[target],ha='center', fontsize=10)
            ax1 = fig.add_subplot(8,6,e+7)
            ax1.axis('off')
            ax1.scatter(xyz[:,0][xyz[:,1] > 0], xyz[:,2][xyz[:,1] > 0],
                      c=f[:,target][xyz[:,1] > 0], s=0.5, norm=norm,
                      cmap=plt.cm.Blues)
            if e == 0:
                ax.text(-420,0,'WT \n Obs.',ha='left',va='center', rotation=90, fontsize=10)
                ax1.text(-420,0,'WT'+ '\n' + 'Sim.',ha='left',va='center', rotation=90, fontsize=10)

    init_p = np.copy(init)
    init_p[:,source] = init_p[:,source]*0

    quad = make_quad(init_p)
    f_p = init_p + quad.dot(w) + bias
    f_p[f_p<0] = 0
    f_p[:,source] = f_p[:,source]*0

    for i in range(prop):
        quad = make_quad(f_p)
        f_p = f_p + quad.dot(w) + bias
        f_p[f_p<0] = 0
        f_p[:,source] = f_p[:,source]*0
    
    position=13 + e_s*6
    for (e,target) in enumerate(genes):
        norm = plt.Normalize(np.min(data[:,target]), np.max(data[:,target]))
        ax = fig.add_subplot(8,6,position)
        ax.axis('off')
        xy = ax.scatter(xyz[:,0][xyz[:,1] > 0], xyz[:,2][xyz[:,1] > 0],
                  c=f_p[:,target][xyz[:,1] > 0], s=0.5, norm=norm,
                  cmap=plt.cm.Blues)
        if e == 0:
            ax.text(-340,0,names[source] + '-', ha='left', va='center', rotation=90, fontsize=10)
        position += 1
plt.show()